# Домашнее задание

### OneHot Encode (3 балла)

Реализовать упрощенный вариант преобразования one_hot_encoding, не используя pd.get_dummies и sklearn.preprocessing.OneHotEncoder

Написать функцию, которая принимает на вход одномерный массив numpy размера (число объектов) со значениями категориального признака и выдает матрицу размера (число объектов х число значений фичи), заполненную как one_hot_encoding.

Бинарные вектора в матрице идут в порядке сортировки значений категориального признака. Те если признак принимает значения b, a, с, то самым левым в one_hot матрице будет вектор для a, а самым правым для c. Это же видно на примерах ниже

In [213]:
import numpy as np

In [331]:
def onehot_encode(x):
    features = sorted(list(set(x)))
    #print(features)
    feature_to_idx = {feature: idx for idx, feature in enumerate(features)}
    #print(features_num)
    one_hot = np.zeros((len(x), len(features)))
    for i in range(len(x)):
        one_hot[i][feature_to_idx[x[i]]] = 1
    
    return one_hot

In [332]:
# пример
x = np.array([1, 2, 3, 3, 2, 1])
print(onehot_encode(x))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [333]:
# пример
x = np.array(['b', 'b', 'a', 'b', 'd', 'b', 'a', 'c', 'c', 'a', 'c', 'd'])
print(onehot_encode(x))

[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


### MinMax Scale (3 балла)

Написать функцию MinMaxScaler "своими руками" 

Функция принимает на вход матрицу признаков и выдает отмасштабированную матрицу

In [334]:
def minmax_scale(X):
    X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
    return X_std

In [335]:
# пример
X = np.random.random((5, 2))
X

array([[0.1813737 , 0.3883839 ],
       [0.82522422, 0.1948972 ],
       [0.36096229, 0.06448306],
       [0.27504181, 0.95982753],
       [0.93215052, 0.24144349]])

In [336]:
# пример
minmax_scale(X)

array([[0.        , 0.36176115],
       [0.85757911, 0.14565806],
       [0.23920369, 0.        ],
       [0.12476159, 1.        ],
       [1.        , 0.19764507]])

In [337]:
from sklearn import preprocessing
sklearn.preprocessing.minmax_scale(X)

array([[0.        , 0.36176115],
       [0.85757911, 0.14565806],
       [0.23920369, 0.        ],
       [0.12476159, 1.        ],
       [1.        , 0.19764507]])

### Работа с признаками (4 балла)

Скачайте датасет из материалов к уроку или по ссылке https://raw.githubusercontent.com/jupiterzhuo/travel-insurance/master/travel%20insurance.csv 


Описание признаков:

* Agency — название страхового агентства
* Agency Type — тип страхового агентства
* Distribution Channel — канал продвижения страхового агентства
* Product Name — название страхового продукта
* Duration — длительность поездки (количество дней)
* Destination — направление поездки
* Net Sales — сумма продаж 
* Commission (in value) — комиссия страхового агентства
* Gender — пол застрахованного
* Age — возраст застрахованного

Ответ:
* Claim — потребовалась ли страховая выплата: «да» — 1, «нет» — 0

Обработайте пропущенные значения и примените написанные выше функции onehot_encode() и minmax_scale().

In [338]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/jupiterzhuo/travel-insurance/master/travel%20insurance.csv")
df.head()

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,81
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,71
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65,AUSTRALIA,-49.5,29.70,NaN,32
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60,AUSTRALIA,-39.6,23.76,NaN,32
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79,ITALY,-19.8,11.88,NaN,41


In [339]:
df.isnull().sum()

Agency                      0
Agency Type                 0
Distribution Channel        0
Product Name                0
Claim                       0
Duration                    0
Destination                 0
Net Sales                   0
Commision (in value)        0
Gender                  45107
Age                         0
dtype: int64

In [340]:
df.replace({"No": 0.0, "Yes": 1.0}, inplace=True)

In [341]:
cat_features_mask = (df.dtypes == "object").values
df_num = df[df.columns[~cat_features_mask]]
df_cat = df[df.columns[cat_features_mask]].replace(np.nan, "")

In [342]:
def onehot_encode_cat_matrix(X):
    X_ans = pd.DataFrame(np.zeros((X.shape[0], 0)))
    for col in X.columns:
        features = X[col].values
        #print(features.shape)
        cur_matrix = pd.DataFrame(onehot_encode(features), columns=sorted(list(set(X[col].values))))
        X_ans = pd.concat([X_ans, cur_matrix], axis=1)
    return X_ans

In [344]:
onehot_cat = onehot_encode_cat_matrix(df_cat)
scaled_num = minmax_scale(df_num)

df_norm = pd.concat((scaled_num, onehot_cat), axis=1)


### Применение линейной регрессии (10 баллов)

Это задача классификации, но её можно решить с помощью линейной регрессии, если округлять предсказанный ответ до целого и выбирать ближайший по значению ответ из множества {0, 1}.

Вынесите признак 'Claim' в вектор ответов и разделите датасет на обучающую и тестовую выборку в соотношении 80 к 20.


In [350]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

y = df.Claim
X = df_norm.drop(columns='Claim')
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

Найдите аналитическое решение для обучающей выборки: обычное и регуляризацией l2. 

In [351]:
# посчитайте аналитическое решение
analitic_w = np.linalg.inv(X.T @ X) @ X.T @ y
analitic_w = analitic_w[:,np.newaxis]

#print(X_train.shape, analitic_w.shape)

X_test @ analitic_w 

C:\Users\avear\AppData\Local\Temp\ipykernel_7484\370019650.py:3: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  analitic_w = analitic_w[:,np.newaxis]


,0
10134,-0.902718
7022,-1.115608
55680,-3.191282
14767,0.766726
23870,-0.918343
...,...
11373,-1.433968
58058,-3.034554
18304,1.816530
54463,-0.765999


In [352]:
# посчитать аналитическое решение с регуляризацией
n, k = X.shape
X_train1 = X
X_train1 = np.hstack((np.ones((n, 1)), X))

#print(X_train1.shape, analitic_w_l2.shape)


l = 1

analitic_w_l2 = np.linalg.inv(X_train1.T @ X_train1 + l * np.eye(k + 1)) @ X_train1.T @ y
analitic_w_l2 = analitic_w_l2[:,np.newaxis][1:]


X_test @ analitic_w_l2 

,0
10134,0.012477
7022,0.011032
55680,0.056921
14767,0.016717
23870,0.012154
...,...
11373,0.014763
58058,0.020768
18304,0.060320
54463,0.016159


Постройте модель LinearRegression и Ridge, применить к тестовой выборке и посчитайте MSE (можно использовать библиотеку sklearn)

In [353]:
# обучите модель линейной регрессии LinearRegression на обучающей выборке, примените к тестовой
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt


lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

lr_l2 = Ridge()
lr_l2.fit(X_train, y_train)
y_pred_l2 = lr_l2.predict(X_test)

In [354]:
# посчитайте MSE, предварительно округлив предсказанные ответы до целого

y_pred = np.round(y_pred)
y_pred_l2 = np.round(y_pred_l2)


mse = mean_squared_error(y_test, y_pred)
mse_l2 = mean_absolute_error(y_test, y_pred_l2)


print(mse, mse_l2)

1.854856144811852e+18 0.014448128848886783
